BLG 453E - Computer Vision<br>Homework 4<br>Muhammed Tolga Cangöz<br>150130024

Part 1

In [6]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import moviepy.editor as mpy

In [ ]:
for k in range(0, 455, 5):
    img1 = plt.imread("DJI_0101/{:05d}.png".format(k))
    img1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    img2 = plt.imread("DJI_0101/{:05d}.png".format(k+5))
    img2 = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)

    img1 /= 255.0
    img2 /= 255.0
    window_size = 5
    threshold = 0.000001
    filterX = np.array([[-1., 1.], 
                        [-1., 1.]])
    filterY = np.array([[-1., -1.], 
                        [1., 1.]])
    filterT = np.array([[1., 1.], 
                        [1., 1.]])
    ws = int(window_size/2)
    
    # Derivatives of the images in the direction of x
    fx = cv2.filter2D(img1, -1, cv2.flip(filterX, -1), borderType=cv2.BORDER_REFLECT)
    # Derivatives of the images in the direction of y
    fy = cv2.filter2D(img1, -1, cv2.flip(filterY, -1), borderType=cv2.BORDER_REFLECT)
    # ft means the difference between the two consecutive images (in time), ft = img2 - img1
    ft = cv2.filter2D(img2, -1, cv2.flip(filterT, -1), borderType=cv2.BORDER_REFLECT) +\
        cv2.filter2D(img1, -1, cv2.flip(-filterT, -1), borderType=cv2.BORDER_REFLECT)
    u, v = np.zeros_like(img1), np.zeros_like(img1)
    
    for i in range(ws, img1.shape[0] - ws):
        for j in range(ws, img1.shape[1] - ws):
            Ix, Iy = fx[i - ws: i + ws + 1, j - ws: j + ws + 1].flatten(), fy[i - ws:i + ws + 1, j - ws:j + ws + 1].flatten()
            b  = ft[i - ws:i + ws + 1, j - ws: j + ws + 1].reshape(window_size*window_size, 1)
            A  = np.transpose(np.vstack((Ix, Iy)))
            # If there is enough information to estimate the flow, the least square method is used
            if np.sort(np.abs(np.linalg.eigvals(A.T.dot(A))))[0] >= threshold:
                u[i,j], v[i,j] = np.linalg.pinv(A.T.dot(A)).dot(A.T).dot(b)

    U = np.abs(u) + np.abs(v)
    thresh_image = cv2.threshold(U, 2, 255, 0)[1].astype(np.uint8)
    a = cv2.findContours(thresh_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    center_points = []
    for c in a[0]:
        M = cv2.moments(c)
        if M["m00"] != 0:
            centerX = int(M["m10"] / M["m00"])
            centerY = int(M["m01"] / M["m00"])
        else:
            continue
        center_points.append([centerY, centerX])
    
    # Mainly, there are two blobs moving in the pictures, the left one and the right one.
    blob1 = center_points[0]
    blob2 = center_points[-1]
    color = (0, 255, 0) # Green color in BGR for arrowed line
    thickness = 5

    start_point = blob1[0], blob1[1]
    end_point = blob1[0] + int(10*v[blob1[0], blob1[1]]), blob1[1] + int(10*u[blob1[0], blob1[1]])
    img2 = plt.imread("DJI_0101/{:05d}.png".format(k+5))
    image = cv2.arrowedLine(cv2.cvtColor(img2, cv2.COLOR_RGB2BGR), start_point[::-1], end_point[::-1], color, thickness)


    start_point = blob2[0], blob2[1]
    end_point = blob2[0] + int(10*v[blob2[0], blob2[1]]), blob2[1] + int(10*u[blob2[0], blob2[1]])
    image = cv2.arrowedLine(cv2.cvtColor(image, cv2.COLOR_RGB2BGR), start_point[::-1], end_point[::-1], color, thickness)
    image[image == 255] = 1

    plt.imsave("part1/{:05d}.png".format(k), (image*255).astype(np.uint8))

In [ ]:
clip = mpy.ImageSequenceClip(os.path.join("part1/"), fps=10)
clip.write_videofile('part1_video.mp4')

Part 2

In [33]:
images_list = []
# Choosing first image as a reference point
reference = plt.imread("DJI_0101/00000.png")
reference_gray = cv2.cvtColor(reference, cv2.COLOR_RGB2GRAY)
reference_gray = cv2.GaussianBlur(reference_gray, (21, 21), 0)


for i in range(1, 460):
    each_frame = plt.imread("DJI_0101/%05d.png" % i)
    
    gray = cv2.cvtColor(each_frame, cv2.COLOR_RGB2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    # Take differences between the reference and the next frame
    difference = cv2.absdiff(gray, reference_gray)
    # Remove noise a bit
    thresh = cv2.threshold(difference, 0.075, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=2)
    
    images_list.append(thresh)

In [34]:
for i in range(len(images_list)):
    plt.imsave("part2/%05d.png" % i, images_list[i], cmap='gray')

In [ ]:
clip = mpy.ImageSequenceClip(os.path.join("part2/"), fps=24)
clip.write_videofile('part2_video.mp4')